In [ ]:
import great_expectations as ge
import yaml
from great_expectations.core.batch import BatchRequest, RuntimeBatchRequest
context = ge.data_context.DataContext()
from pprint import pprint

In [58]:
# Check the batch
path = "../0_raw data/raw_ohlcv.csv"
suite = context.get_expectation_suite(expectation_suite_name='raw_data_suite')
runtime_batch_request = RuntimeBatchRequest(
    datasource_name="raw_data",
    data_connector_name="default_runtime_data_connector_name",
    data_asset_name="raw ohlcv",
    runtime_parameters={"path": path},
    batch_identifiers={"default_identifier_name":1,},
    batch_spec_passthrough={"reader_method": "read_csv","reader_options": {"sep": ",","header": 0}}
)
my_validator = context.get_validator(batch_request=runtime_batch_request,expectation_suite=suite)
my_validator.head(n_rows=5,fetch_all=False)


,Stock,Date,Open,High,Low,Close,Adjusted_close,Volume
,0LCV.LSE,2018-01-15 00:00:00+00:00,41.3,41.3,41.3,41.3,41.3,0.0
0,0LCV.LSE,2018-01-16 00:00:00+00:00,41.8500,42.2863,41.8500,41.8500,41.8500,8835.0
1,0LCV.LSE,2018-01-17 00:00:00+00:00,42.7640,42.7640,42.7640,42.7640,42.7640,2040.0
2,0LCV.LSE,2018-01-18 00:00:00+00:00,44.3500,44.3500,44.3500,44.3500,44.3500,9083.0
3,0LCV.LSE,2018-01-19 00:00:00+00:00,44.8102,44.9000,44.8102,44.8102,44.8102,4670.0
4,0LCV.LSE,2018-01-22 00:00:00+00:00,46.2300,46.2300,46.2300,46.2300,46.2300,873.0


In [ ]:
# Create the checkpoint
checkpoint_name = "1st_checkpoint" 
yaml_config = f"""
name: {checkpoint_name}
config_version: 1.0
class_name: SimpleCheckpoint
run_name_template: "%Y%m%d-%H%M%S-my-run-name-template"
validations:
  - batch_request:
      datasource_name: raw_data
      data_connector_name: default_inferred_data_connector_name
      data_asset_name: raw_ohlcv.csv
      data_connector_query:
        index: -1
    expectation_suite_name: raw_data_suite
"""
# Print out the names of your Datasources, Data Connectors and Data Assets
pprint(context.get_available_data_asset_names())
context.list_expectation_suite_names()
# Run a check on the checkpoint configuration
checkpoint = context.test_yaml_config(yaml_config=yaml_config)
# Review the checkpoint
print(checkpoint.get_substituted_config().to_yaml_str())
# Add the checkpoint to the checkpoint store
context.add_checkpoint(**yaml.load(yaml_config))

In [ ]:
# Run the checkpoint
results = context.run_checkpoint(checkpoint_name=checkpoint_name)
# Print the results
print(results)
# Open the data docs
context.open_data_docs()